In [1]:
%pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found

In [2]:
%pip install captum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.2 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import warnings
import pandas as pd

import bitsandbytes as bnb
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig

from collections import defaultdict
from datasets import Dataset
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
import torch
import string

from captum.attr import (
    FeatureAblation, 
    ShapleyValues,
    LayerIntegratedGradients, 
    LLMAttribution, 
    LLMGradientAttribution, 
    TextTokenInput, 
    TextTemplateInput,
    ProductBaselines,
)

# Ignore warnings due to transformers library
warnings.filterwarnings("ignore", ".*past_key_values.*")
warnings.filterwarnings("ignore", ".*Skipping this token.*")

2025-08-01 00:48:29.080683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754009309.247326      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754009309.296051      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
def load_model(model_name, bnb_config):
    n_gpus = torch.cuda.device_count()
    max_memory = "10000MB"

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        num_labels=3,
        device_map="auto",  # dispatch efficiently the model on the available ressources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)

    # Needed for LLaMA tokenizer
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

In [ ]:
from huggingface_hub import login

login(new_session=False, # Won’t request token if one is already saved on machine
write_permission=True, # Requires a token with write permission
token="", # The name of your token
add_to_git_credential=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)
Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [7]:
model_name = "ConditionalNLI/exp8_llama3.2" 
bnb_config = create_bnb_config()
model, tokenizer = load_model(model_name, bnb_config)

adapter_config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.safetensors:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

In [8]:
model.config.pad_token_id = model.config.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
model.config.problem_type = "single_label_classification"

In [9]:
test_df4 = pd.read_excel('/kaggle/input/confer-extension/Part4B_Dataset/4B_Type4.xlsx')
test_df4

,premise,hypothesis,gold_label,trigger,key_phrase,type
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,his wind tunnel,related
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,his content management application,related
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,related
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,his wetsuit,related
4,"If Noah is a pastry chef, he'll pipe frosting ...",Noah has a pastry bag.,E,possessive,his pastry bag,related
...,...,...,...,...,...,...
292,"If Christina is a makeup artist, John's friend...",Christina has a makeup sponge.,N,possessive,John's friend,unrelated
293,"If Rahim is a clergyman, John's friend will ca...",Rahim has a holy book.,N,possessive,John's friend,unrelated
294,"If James is a police officer, John's friend wi...",James has a gun.,N,possessive,John's friend,unrelated
295,"If Maya is a piano teachis, John's friend will...",Maya has a metronome.,N,possessive,John's friend,unrelated


In [10]:
test_df5a = pd.read_excel('/kaggle/input/confer-extension/Part4B_Dataset/4B_Type5A.xlsx')
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase,type
0,"If Jessica attends the conference, she'll neve...",Jessica has watched a movie by Nolan before.,E,again,watch a movie by Nolan again,related
1,"If Emily passes her driving test, she'll never...",Emily has read a book by Dan Brown before.,E,again,read a book by Dan Brown again,related
2,"If Scarlett finishes her work early, she'll ne...",Scarlett has eaten a traditional Swedish dish ...,E,again,eat a traditional Swedish dish again,related
3,"If Chloe finishes her work early, she'll never...",Chloe has sung a pop song before.,E,again,sing a pop song again,related
4,"If Charlotte passes her driving test, she'll n...",Charlotte has listened to a pop song before.,E,again,listen to a pop song again,related
...,...,...,...,...,...,...
1595,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,N,again,obtain a job position,no again
1596,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,N,again,convey a confidential message,no again
1597,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,N,again,manage to keep a secret,no again
1598,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,N,again,cherish a moment in peace,no again


In [11]:
test_df5p = pd.read_excel('/kaggle/input/confer-extension/Part4B_Dataset/4B_Type5P.xlsx')
test_df5p

,premise,hypothesis,gold_label,trigger,key_phrase,type
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,related
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,related
2,"If Scarlett finishes her work early, her husba...",Scarlett has a husband.,E,possessive,her husband,related
3,"If Chloe finishes his work early, his colleagu...",Chloe has colleagues.,E,possessive,his colleagues,related
4,"If Charlotte passes her driving test, her pare...",Charlotte has parents.,E,possessive,her parents,related
...,...,...,...,...,...,...
1195,"If Dorothy car breaks down, Tina's friend will...",Dorothy has a brother.,N,possessive,Tina's friend,unrelated
1196,"If Susan finishes his work early, Tina's frien...",Susan has a colleague.,N,possessive,Tina's friend,unrelated
1197,"If Margaret finishes work early, Tina's friend...",Margaret has a wife.,N,possessive,Tina's friend,unrelated
1198,"If Betty misses the bus, Tina's friend will dr...",Betty has a brother.,N,possessive,Tina's friend,unrelated


In [12]:
sample_test_df = test_df4.iloc[:4].reset_index(drop=True)
sample_test_df

,premise,hypothesis,gold_label,trigger,key_phrase,type
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,his wind tunnel,related
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,his content management application,related
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,related
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,his wetsuit,related


In [13]:
# test_df = sample_test_df
test_df = test_df5a
possessive_trigger = False

In [14]:
test_ds = Dataset.from_pandas(test_df)
test_ds

Dataset({
    features: ['premise', 'hypothesis', 'gold_label', 'trigger', 'key_phrase', 'type'],
    num_rows: 4
})

In [15]:
columns_to_keep = ['index', 'premise', 'hypothesis']
test_ds = test_ds.remove_columns([col for col in test_ds.column_names if col not in columns_to_keep])
test_ds

Dataset({
    features: ['premise', 'hypothesis'],
    num_rows: 4
})

In [16]:
def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True, max_length=256, padding='max_length')

In [17]:
tokenized_test_ds = test_ds.map(tokenize_function, batched=True)
tokenized_test_ds.set_format("torch")
tokenized_test_ds

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'input_ids', 'attention_mask'],
    num_rows: 4
})

In [18]:
tokenized_test_ds = tokenized_test_ds.remove_columns(["premise", "hypothesis"])
tokenized_test_ds

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 4
})

In [19]:
# Predicting Labels on Dataset
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=256, return_tensors="pt")
dataloader = DataLoader(tokenized_test_ds, batch_size=32, collate_fn=data_collator)

model.eval()
predictions = []

with torch.no_grad():
    for batch in dataloader:
        batch = {k: v.to("cuda") for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.cpu().tolist())

test_df['predicted_label'] = predictions

In [20]:
test_df['expected_logits'] = test_df['gold_label'].map({'C':[1,0,0],'N':[0,1,0],'E':[0,0,1]})
test_df

,premise,hypothesis,gold_label,trigger,key_phrase,type,predicted_label,expected_logits
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,his wind tunnel,related,2,"[0, 0, 1]"
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,his content management application,related,2,"[0, 0, 1]"
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,related,2,"[0, 0, 1]"
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,his wetsuit,related,2,"[0, 0, 1]"


## Integrated Gradients

In [21]:
import torch.nn as nn

class LogitWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.logits
        logits = logits.unsqueeze(0) # add third dimension 
        output.logits = logits
        return output

In [22]:
wrapped_model = LogitWrapper(model)
lig = LayerIntegratedGradients(wrapped_model, model.model.embed_tokens)
llm_attr = LLMGradientAttribution(lig, tokenizer)
skip_tokens = ['<|begin_of_text|>','<|end_of_text|>']

In [23]:
def compute_attrs(row):

    premise = row['premise']
    hypothesis = row['hypothesis']
    target = torch.tensor(row['expected_logits'])    
    
    inp = TextTokenInput(
        premise + tokenizer.eos_token + hypothesis,
        tokenizer,
        skip_tokens=skip_tokens,
    )
    attr_res = llm_attr.attribute(inp, target=target, skip_tokens=skip_tokens)
    result = {
        'tokens': attr_res.input_tokens,
        'attrs': attr_res.seq_attr
    }
    return result

In [36]:
def reformat_output(result):

    i = 0 # position counter
    token_gradients = defaultdict(float)
    print(result['tokens'])
    for token, grad in zip(result['tokens'], result['attrs']):
        if token.startswith('▁'):
            token = token[1:]
        
        token_gradients[(token,i)] += grad.item()
        i += 1 

    token_gradients = dict(token_gradients)
    tokens = [tok[0] for tok in list(token_gradients.keys())]
    word_gradients = {}
    print(token_gradients.keys())

    for i, token in enumerate(tokens):
        word = tokens[i]
        if word.startswith('Ġ'):
            word = word[1:]
        count = 1 
        word_gradient = token_gradients[(token,i)]
        if word in ['[SEP]','[CLS]']:
             word_gradients[(word,i)] = word_gradient
             continue # do not merge with next token
        while (i+1)<len(tokens) and tokens[i+1] not in string.punctuation and not tokens[i+1].startswith('Ġ') and not tokens[i+1] == '[SEP]':
            # part of same word, add gradients together and then average out 
            i += 1 # move to next word
            word += tokens[i] # concatente to restore word 
            word_gradient += token_gradients[(tokens[i],i)]
            count += 1
        # end of word 
        word_gradients[(word,i)] = word_gradient/count # averaging
    
    return word_gradients

In [25]:
def get_average_gradient(result):
     # average gradient per token
    return float(sum(result['attrs'])/len(result['tokens']))

In [33]:
def compute_trigger_gradients(row):
    
    gradients = 0
    count = 0
    trigger_words = []

    # split phrases into individual words
    trigger_words = row['key_phrase'].split()

    # get gradient of all words in this example
    word_gradients = row['word_gradients']
    word_gradients_words = [key[0] for key in list(word_gradients.keys())] # extracting word portion of keys
    
    word_index_mapping = {}
    
    for word, index in list(word_gradients.keys()):
        word_index_mapping.setdefault(word,index) # set the first value (premise value) as the index
    
    for word in trigger_words:
        if word in word_gradients_words:
            word_index = word_index_mapping[word]
            gradients += word_gradients[(word,word_index)]
            count += 1 
        elif word.lower() in word_gradients_words:
            word_index = word_index_mapping[word.lower()]
            gradients += word_gradients[(word.lower(),word_index)]
            count += 1 
        else:
            print(f"{row.name}: {word} not found!")

    # compute avg gradient of trigger tokens
    return gradients/count if count > 0 else 0

In [27]:
def compute_trigger_word_ranking(word_gradients,possessive_trigger):

    sorted_dict = dict(sorted(word_gradients.items(), key=lambda item: item[1],reverse=True))
    sorted_dict = {k: v for k,v in sorted_dict.items() if k[0] not in ['[CLS]','[SEP]'] and k[0] not in string.punctuation}
    word_gradients_df = pd.DataFrame.from_dict(sorted_dict, orient='index', columns=['Gradient'])
    word_gradients_df['Ranking'] = range(1, len(word_gradients_df) + 1)
    word_gradients_df['Percentile Ranking'] = word_gradients_df['Ranking'].apply(lambda x: x / len(word_gradients_df))

    word_gradients_words = [key[0] for key in list(word_gradients.keys())] # extracting word portion of keys 

    if possessive_trigger:
        if 'his' in word_gradients_words:
            key_trigger_word = 'his'
        elif 'her' in word_gradients_words:
            key_trigger_word = 'her'
        elif 'their' in word_gradients_words:
            key_trigger_word = 'their'
    else:
        key_trigger_word = 'again'

    try:
        key_trigger_word_rank = word_gradients_df.loc[key_trigger_word]['Ranking']
        key_trigger_word_percent_rank = word_gradients_df.loc[key_trigger_word]['Percentile Ranking']
    except:
        key_trigger_word_rank = None 
        key_trigger_word_percent_rank = None
        print(f"Trigger word not found! - {key_trigger_word}")

    return pd.Series([key_trigger_word_rank, key_trigger_word_percent_rank])

In [37]:
test_df['attr_result'] = test_df.apply(compute_attrs, axis=1)
test_df.to_csv('part4b_type5a_llama_raw.csv')
test_df['word_gradients'] = test_df['attr_result'].apply(reformat_output)
test_df['average_gradient'] = test_df['attr_result'].apply(get_average_gradient)
test_df['trigger_gradient'] = test_df.apply(compute_trigger_gradients,axis=1)
test_df['predicted_label'] = test_df['predicted_label'].map({0:'C',1:'N',2:'E'})
test_df[['trigger_word_ranking','trigger_word_percentile_ranking']] = test_df['word_gradients'].apply(compute_trigger_word_ranking,args=(possessive_trigger,))
test_df

/usr/local/lib/python3.11/dist-packages/captum/attr/_core/llm_attr.py:840: UserWarning: Pretty tokens length 0 != ids length 3! Falling back to naive decoding logic.
  _convert_ids_to_pretty_tokens(target_tokens, self.tokenizer),


['If', 'ĠLiam', 'Ġis', 'Ġan', 'Ġaerospace', 'Ġengineer', ',', 'Ġhe', "'ll", 'Ġtest', 'Ġthe', 'Ġdesigns', 'Ġin', 'Ġhis', 'Ġwind', 'Ġtunnel', '.', 'L', 'iam', 'Ġhas', 'Ġa', 'Ġwind', 'Ġtunnel', '.']
dict_keys([('If', 0), ('ĠLiam', 1), ('Ġis', 2), ('Ġan', 3), ('Ġaerospace', 4), ('Ġengineer', 5), (',', 6), ('Ġhe', 7), ("'ll", 8), ('Ġtest', 9), ('Ġthe', 10), ('Ġdesigns', 11), ('Ġin', 12), ('Ġhis', 13), ('Ġwind', 14), ('Ġtunnel', 15), ('.', 16), ('L', 17), ('iam', 18), ('Ġhas', 19), ('Ġa', 20), ('Ġwind', 21), ('Ġtunnel', 22), ('.', 23)])
['If', 'ĠBill', 'Ġis', 'Ġa', 'Ġcontent', 'Ġstrategist', ',', 'Ġhe', "'ll", 'Ġmanage', 'Ġeverything', 'Ġin', 'Ġhis', 'Ġcontent', 'Ġmanagement', 'Ġapplication', '.', 'Bill', 'Ġhas', 'Ġa', 'Ġcontent', 'Ġmanagement', 'Ġapplication', '.']
dict_keys([('If', 0), ('ĠBill', 1), ('Ġis', 2), ('Ġa', 3), ('Ġcontent', 4), ('Ġstrategist', 5), (',', 6), ('Ġhe', 7), ("'ll", 8), ('Ġmanage', 9), ('Ġeverything', 10), ('Ġin', 11), ('Ġhis', 12), ('Ġcontent', 13), ('Ġmanagement', 1

,premise,hypothesis,gold_label,trigger,key_phrase,type,predicted_label,expected_logits,attr_result,word_gradients,average_gradient,trigger_gradient,trigger_word_ranking,trigger_word_percentile_ranking
0,"If Liam is an aerospace engineer, he'll test t...",Liam has a wind tunnel.,E,possessive,his wind tunnel,related,NaN,"[0, 0, 1]","{'tokens': ['If', 'ĠLiam', 'Ġis', 'Ġan', 'Ġaer...","{('If', 0): 1.275778777198866e-05, ('Liam', 1)...",0.001006,0.000751,None,None
1,"If Bill is a content strategist, he'll manage ...",Bill has a content management application.,E,possessive,his content management application,related,NaN,"[0, 0, 1]","{'tokens': ['If', 'ĠBill', 'Ġis', 'Ġa', 'Ġcont...","{('If', 0): 0.0002884181449189782, ('Bill', 1)...",-0.000913,0.000465,None,None
2,"If Steve is a conservation officer, he'll set ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,related,NaN,"[0, 0, 1]","{'tokens': ['If', 'ĠSteve', 'Ġis', 'Ġa', 'Ġcon...","{('If', 0): 0.0006778022507205606, ('Steve', 1...",0.000426,0.000509,None,None
3,"If Matt is a scuba diver, he'll bring his wets...",Matt has a wetsuit.,E,possessive,his wetsuit,related,NaN,"[0, 0, 1]","{'tokens': ['If', 'ĠMatt', 'Ġis', 'Ġa', 'Ġsc',...","{('If', 0): 0.00021063859458081424, ('Matt', 1...",0.000823,-0.000349,None,None


In [ ]:
test_df.to_csv('part4b_type5a_llama_results_new.csv')